In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import sqlite3 as sl3


In [ ]:
JupyterDash.infer_jupyter_proxy_config()  #  Need this for binder to work.

In [2]:
conn = sl3.connect('fash.db')
qry = "select * from sales"
df = pd.read_sql(qry, con=conn)

prod_des = 100 *  \
            pd.crosstab(index = df['product'], columns=df['designer'],margins= False,normalize= 'index')




app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Fashion Demo"),
     html.Label([
        "Choose brand",
        dcc.Dropdown(
            id='brand', clearable=False,
            value='Zegna', options=[{'label': c, 'value': c} for c in set(df['designer'])
            ]),
    html.Label([
        "Choose price cutoff",
        dcc.Dropdown(
            id='prc', clearable=False,
            value=10.00, options=[{'label':10.00,'value':10.00},{'label':20.00,'value':20.00}
            ]),
    html.Div(children=[
        dcc.Graph(id="graph")
    ]),   
    ]),
])  ])  

In [3]:
@app.callback(Output('graph', 'figure'),Input("brand", "value"),Input("prc","value") )   
def update_figure(brand,prc):
    fash = df[(df['designer']== brand) & (df['price'] > float(prc))]
    fig = make_subplots(rows=1,cols=2,subplot_titles=('Plot 1','Plot 2'))
    fig.add_trace(go.Bar(x=fash['product'],y=fash['day']), row=1, col=1)
    fig.add_trace(go.Bar(x=fash['product'],y=fash['store'],orientation='h'), row=1, col=2)
    return fig
   
    


app.run_server(mode='inline')